In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import pandas as pd
# from transformers import Trainer, TrainingArguments, DataCollatorWithPadding
from datasets import load_dataset,DatasetDict,load_from_disk,Dataset
# from transformers import AutoTokenizer,AutoModelForSequenceClassification
# import evaluate
# import torch
import nltk
import random
import numpy as np
label2id = {
    "AddToPlaylist": 0,
    'BookRestaurant': 1,
    'GetWeather': 2,
    'PlayMusic': 3,
    'RateBook': 4,
    'SearchCreativeWork': 5,
    'SearchScreeningEvent': 6,
}

In [3]:
language = 'tr'

if language == 'en':
    ds_orig = load_dataset('benayas/snips')
else: ds_orig = load_from_disk("/snips_dataset/snips_"+language+".hf")
df_test = pd.DataFrame(ds_orig['test'])
# df_test = df_test[700:]
df_test['word_count'] = df_test['text'].apply(lambda x: len(str(x).split()))

# Calculate min and max word count
min_word_count = df_test['word_count'].min()
max_word_count = df_test['word_count'].max()
mean_word_count = df_test['word_count'].mean()
print(f"Minimum number of words: {min_word_count}")
print(f"Maximum number of words: {max_word_count}")
print(f"Average number of words: {int(mean_word_count)}")
df_test = df_test.drop(columns = ['word_count'])

Minimum number of words: 2
Maximum number of words: 19
Average number of words: 7


In [8]:
dir+filename

'/Noise Dics/wiki_typos_de_p320000_l15023.pkl'

### Create noisy test set using Wikitypo

In [4]:
import pickle
import os
dir = '/Noise Dics/'
for file in os.listdir(dir):
        filename = os.fsdecode(file)
        if language in filename:
            print(filename)
            with open(dir+filename,'rb') as f:
                ds_t = pickle.load(f)

wiki_typos_tr_p320000_l4064.pkl


In [5]:
df = df_test.copy()

# Function to replace a percentage of tokens in a sentence
def replace_noisy(sentence, percent_to_replace=30 , max_aug=10):

    tokens = nltk.word_tokenize(sentence)
    num_tokens_to_replace = int((len(tokens)-1) * (percent_to_replace / 100))
    index_list = random.sample(range(0, len(tokens) ),len(tokens)-1)
    cc =0
    for i in index_list:
        if (cc == num_tokens_to_replace) or (cc== max_aug):
            break
        index_to_replace = i
        original_token = tokens[index_to_replace]
        random.shuffle(ds_t)
        for original, replacement in ds_t:
            if original_token.startswith(original):
                tokens[index_to_replace] = original_token.replace(original, replacement.lower())
                cc = cc +1
                break
    
    res=' '.join(tokens)
    return res

# Applying the custom function to the 'sentence' column with a specified percentage (e.g., 30%)
df['text'] = df['text'].apply(lambda x: replace_noisy(x, percent_to_replace=20,max_aug = 4))
df['text']


0                Estelle ilebahar çalma listemde olmalı .
1       Jovino santas Neto'nun albümüü Confidence Boos...
2       Ila'nın çalma listsine ekle A Mi Manera Recopi...
3       Sam Sparro'yu Beach Vibes adlı çalma listeeme ...
4       Lütfen bu şarkıyı Urban Hits çalma lısteme ekle .
                              ...                        
1395         Gençlik Ruhu'nu oynatan biri sinema var mı ?
1396    Mahalledeki animasyon filmlerinin filmi saatle...
1397     Great Escape Tiyatrosu'nda film programı dedir ?
1398    Dipson Tiyatrosu'ndaki Cheers fr Miss Bishop'u...
1399    Married to the Enemy 2'yi sinemada izlemek itt...
Name: text, Length: 1400, dtype: object

In [6]:
ds_typo = Dataset.from_pandas(df , features = ds_orig['train'].features,preserve_index=False)
ds_typo.save_to_disk("snips_WikiTypos/"+language+"_noisy.hf")

Saving the dataset (0/1 shards):   0%|          | 0/1400 [00:00<?, ? examples/s]

In [95]:
path = '/Snips_noisy_dataset/'+language+'_typos'
ds_typo = dataset_noisy(path,ds_orig)
pd.DataFrame(ds_typo)

,text,category
0,I'd like to have this track onto my Classical ...,AddToPlaylist
1,Add the album to my Flow Español playlist.,AddToPlaylist
2,add biging now to my Young at Heart playlist,AddToPlaylist
3,add this song by Too Poetic to my Piano Ballad...,AddToPlaylist
4,Add this album to Old School Death Metal,AddToPlaylist
...,...,...
695,Is any cinema playing The Spirit of Youth,SearchScreeningEvent
696,What ore the movie times fur animated movies i...,SearchScreeningEvent
697,What's the movie scadule at Great Escape Theatres,SearchScreeningEvent
698,Show th times for Cheers for Miss Bishop at Di...,SearchScreeningEvent


In [33]:
def dataset_noisy(path,ds_orig):
    df_typo = pd.read_table(path+'/seq.in',names=['text'])
    df_typo_label = pd.read_table(path+'/label',names=['category'])
    df_typo = pd.concat([df_typo,df_typo_label], axis=1)
    ds_typo = Dataset.from_pandas(df_typo , features = ds_orig['train'].features,preserve_index=False)
    return ds_typo